In [ ]:
%pip install langchain
%pip install ctransformers
%pip install sentence-transformers
%pip install pinecone-client
%pip install pypdf
%pip install python_dotenv
%pip install langchain-pinecone
%pip install langchain-community
%pip install langchain-core
%pip install langchain-huggingface

In [1]:
from langchain.chains import RetrievalQA 
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_pinecone import PineconeVectorStore as langchainpinecone
from langchain_community.document_loaders import PyPDFLoader,DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import PromptTemplate
from langchain_community.llms import CTransformers
from pinecone import Pinecone
import os

a:\ML\AI_ML_Code\PDF-ChatbotUsing_RAG\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Enter PINECONE_API_KEY below

In [ ]:
PINECONE_API_KEY="Enter your pinecone KEY here"

In [3]:
def load_pdf(data):
    loader=DirectoryLoader(data,
                           glob="*.pdf",
                           loader_cls=PyPDFLoader)
    documents=loader.load()
    return documents

In [4]:
base_path = os.getcwd()
one_level_up = os.path.dirname(base_path)
full_path = os.path.join(one_level_up,"data")
print(full_path)

a:\ML\AI_ML_Code\PDF-ChatbotUsing_RAG\data


In [5]:
extracted_data=load_pdf(full_path)

In [6]:
def text_spit(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [7]:
text_chunks=text_spit(extracted_data)
print("length of my chunk :",len(text_chunks))

length of my chunk : 5860


In [8]:
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [9]:
embeddings=download_hugging_face_embeddings()

In [10]:
os.environ["PINECONE_API_KEY"] =PINECONE_API_KEY

## Enter inder_name below

In [ ]:
pc=Pinecone(api_key=PINECONE_API_KEY)
index_name="ENTER your index name here by copying or same as it is"
docsearch=langchainpinecone.from_texts([t.page_content for t in text_chunks],embedding=embeddings,index_name=index_name)

In [ ]:
prompt_templete="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [ ]:
PROMPT=PromptTemplate(template=prompt_templete,input_variables=["context","question"])
chain_type_kwargs={"prompt":PROMPT}

In [ ]:
model_path = os.path.join(one_level_up,"model/llama-2-7b-chat.ggmlv3.q4_0.bin")

llm=CTransformers(model=model_path,
                  model_type="llama",
                  config={"max_new_tokens" :512,
                          "temperature" : 0.8})

In [ ]:
qa=RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=docsearch.as_retriever(search_kwargs={"k":2}),
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs)

In [ ]:
while True:
    user_input=input(f"Input Prompt : ")
    if (user_input=="stop"):
        break
    result=qa.invoke({"query":user_input})
    print("Response : ",result["result"])

Response :  Acne is a common skin condition characterized by the formation of comedones (blackheads and whiteheads), pimples, and inflamed red nodules on the face, upper back, and chest. Acne can result from a combination of factors, including hormonal changes, excess oil production, clogged pores, and bacterial infections.
